In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Price Volume data retrieval using Yahoo Finance API

Target tickers:
- S&P 500 Index (SPY)
- VIX Index (^VIX)
- Gold (GC=F)
- Brent Crude (BZ=F)
- WTI (CL=F)

In [2]:
# !apt-get install -y build-essential
# !apt-get install -y wget
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib/
# !./configure --prefix=/usr
# !make
# !make install
!pip uninstall -y numpy
!pip install numpy == 1.20.3
!pip install TA-Lib

import yfinance as yf
import talib

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
zsh:1: = not found
  Using cached numpy-1.26.0-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.0 which is incompatible.


In [14]:
# Create price extractor

class price_volume_data:
    """A class for retrieving and analyzing price and volume data."""
    def __init__(self, tickers, start_date, end_date):
        """
        Initializes a new instance of the price_volume_data class.

        Args:
            tickers (list): A list of tickers for the assets to retrieve data for.
            start_date (str): The start date for the data retrieval (format: 'YYYY-MM-DD').
            end_date (str): The end date for the data retrieval (format: 'YYYY-MM-DD').
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.pv, self.shares = self.__retrieve_data__()
        
        #self.lp, self.ret = self.__get_daily_returns__()
        
    def __retrieve_data__(self):
        """
        Retrieves price and volume data for the given tickers from Yahoo Finance API.

        Returns:
            dict: A dictionary containing the price and volume data for each ticker.
        """

        data, failed = {}, []
        shares_outstanding = []
        for ticker in self.tickers:
            try:
                df = yf.Ticker(ticker).history(start = self.start_date, end = self.end_date)
                shares_outstanding.append(yf.Ticker(ticker).info['sharesOutstanding'])   
                
                data[ticker] = df
                
            except:
                failed.append(ticker)
                
        if failed:
            print('Tickers failed to extract: ', failed)
        
        return data, shares_outstanding
    
    def __get_daily_returns__(self):
        """
        Computes daily returns for the retrieved price data.

        Returns:
            pd.DataFrame: A DataFrame containing the daily price data.
            pd.DataFrame: A DataFrame containing the daily price returns.
        """
        df = pd.DataFrame()
        
        for ticker in self.pv:
            if df.empty:
                df = pd.DataFrame(self.pv[ticker]['Close'].copy())
            else:
                df = df.merge(self.pv[ticker]['Close'], how='outer',left_index=True, right_index=True)

        df.columns = self.tickers
        df = df.fillna(method = 'ffill')
        
        return df, df/df.shift(1) - 1
    
    def get_info(self, ticker = None):
        """
        Prints information about the price and volume data for the specified ticker.
        If no ticker is provided, prints information for all tickers.

        Args:
            ticker (str, optional): The ticker symbol for the asset. Defaults to None.
        """
        if ticker:
            self.pv[ticker].info()
        
        else:
            for ticker in self.pv:
                self.pv[ticker].info()
                
        return None
    
    def plot_price_trends(self, normalize = True, reference = -1):
        """
        Plots the close prices of selected assets over time.

        Args:
            normalize (bool, optional): Indicates whether to normalize the prices. Defaults to True.
            reference (int, optional): Reference point for normalization. Defaults to -1.
        """
        f, ax = plt.subplots(1,1,figsize=(12,5))
        # plot prices
        for key in self.pv:
            last_price = self.pv[key].loc[:, 'Close'].copy()
            if normalize:
                if reference == -1:
                    last_price /= last_price.iloc[0]
                elif reference == 0:
                    midpt = int(last_price.shape[0]/2)
                    last_price /= last_price.iloc[midpt]
                else:
                    last_price /= last_price.iloc[-1]
            last_price.plot(ax = ax, label = key)
        
        ax.title.set_text('Close Prices of selected assets')
        ax.set(ylabel = 'Normalized Price to start of observation')
        ax.legend()
        plt.show()
        
    def plot_corr(self, method='pearson', output = False, series = 'price'):
        """
        Plots the correlation matrix between asset prices or returns.

        Args:
            method (str, optional): The correlation method to use. Defaults to 'pearson'.
            output (bool, optional): Indicates whether to return the correlation matrix. Defaults to False.
            series (str, optional): Indicates whether to use 'price' or 'returns'. Defaults to 'price'.

        Returns:
            pd.DataFrame or None: The correlation matrix if output=True, None otherwise.
        """
        corr_matrix = self.lp.corr(method) if series == 'price' else self.ret.corr(method)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        f, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(corr_matrix, mask=mask, vmin=-1, vmax=1, annot=True)
        
        return corr_matrix if output else None
        
        
    def get_technical_indicators(self, indicators = None, normalize = True):
        """
        Computes and adds various technical indicators to the price data.

        Args:
            indicators (list, optional): A list of tuples specifying the indicators and their parameters.
                Defaults to None.
            normalize (bool, optional): Indicates whether to normalize the indicator values. Defaults to True.
        """
        if not indicators:
            indicators = [('SMA', 5), ('SMA', 14), ('ATR', 5), ('ATR', 14), ('ADX', 5),
                         ('ADX', 14), ('RSI', 5), ('RSI', 14), ('MACD', None), ('BBANDS', None)]
        
        for data in self.pv:
            df = self.pv[data]
            for indicator, param in indicators:
                
                
                if indicator == 'SMA':
                    df[f'{indicator}_{param}'] = talib.SMA(df['Close'], param)

                elif indicator == 'ATR':
                    df[f'{indicator}_{param}'] = talib.ATR(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'ADX':
                    df[f'{indicator}_{param}'] = talib.ADX(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'RSI':
                    df[f'{indicator}_{param}'] = talib.RSI(df['Close'], param) 
                    
                elif indicator == 'MACD':
                    df[f'{indicator}'], df[f'{indicator}_signal'], _ = talib.MACD(df['Close']) 
                    
                    
                elif indicator == 'BBANDS':
                    df[f'{indicator}_upp'], df[f'{indicator}_mid'], df[f'{indicator}_low'] = talib.BBANDS(df['Close'])
                    
                    
                else:
                    print('Indicators not built into the class. Please run TALIB manually!')
            
            df = df.replace([np.inf, -np.inf], np.nan, inplace = True)
            
        return None              

In [15]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM
from keras.regularizers import L1, L1L2, L2
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

import numpy as np
import tensorflow as tf
import random as python_random

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

def reset_seeds(SEED = 624):
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

tf.config.experimental.enable_op_determinism()

In [16]:
class LSTM_predictor:
    """A class for predicting time series using LSTM (Long Short-Term Memory) models."""
    def __init__(self, X, Y):
        """
        Initializes a new instance of the LSTM_predictor class.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.
        """
        self.X, self.Y = self.__clean_data__(X, Y)
        self.features_size = X.shape[1]
        self.obs_size = X.shape[0]
        
        
    def __clean_data__(self, X, Y):
        """
        Cleans the input feature matrix and target values by removing rows with NaN values.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.

        Returns:
            ndarray: The cleaned input feature matrix.
            ndarray: The cleaned target values.
        """
        Z = np.concatenate((X, Y), axis = 1)
        Z = Z[~np.isnan(Z).any(axis=1), :]
        return Z[:, :-1], Z[:, -1]
        
    def prepare_data(self, train_test_split = 0.8, timestep = 1, look_forward = 1, inplace = False, scaled = True):
        """
        Prepares the data for training and testing the LSTM model.

        Args:
            train_test_split (float, optional): The fraction of data to be used for training.
                Defaults to 0.8.
            timestep (int, optional): The number of time steps to consider for each sample.
                Defaults to 3.
            inplace (bool, optional): Indicates whether to store the prepared data in the class
                instance or return it. Defaults to False.
            scaled (bool, optional): Indicates whether to scale the data. Defaults to True.

        Returns:
            tuple or None: If inplace=True, returns None. Otherwise, returns a tuple containing
            the prepared training and testing data as (X_train, Y_train, X_test, Y_test).
        """
        split_idx = int(self.obs_size*0.8)
        train_set = self.X[:-4-look_forward,:], self.Y[timestep+look_forward:-4]
        test_set = self.X[-4-timestep+1:,:], self.Y[-4+look_forward:]
        print(test_set[0])
        
        # Scale factors
        mean_X = np.mean(train_set[0], axis = 0)
        std_X = np.std(train_set[0], axis = 0)
        self.__scale_X__ = (mean_X, std_X)
        
        mean_Y = np.mean(train_set[1], axis = 0)
        std_Y = np.std(train_set[1], axis = 0)
        self.__scale_Y__ = (mean_Y, std_Y)
        
        sampled = []

        for X, Y in [train_set, test_set]:
            X_sampled, Y_sampled = [], []
            #print(X.shape)
            #print(Y.shape)
            print(len(X))

            for i in range(len(X)-timestep-look_forward+1):
                print(i)

                X_sampled.append((X[i:i+timestep] - self.__scale_X__[0])/self.__scale_X__[1])
                Y_sampled.append((Y[i] - self.__scale_Y__[0])/self.__scale_Y__[1])
            
            sampled.append((np.array(X_sampled),np.array(Y_sampled)))
        
        if inplace:
            self.X_train, self.Y_train = sampled[0]
            self.X_test, self.Y_test = sampled[1]
            print("x_train after prepare_data has shape of {}".format(self.X_train.shape))
            print("x_test after prepare_data has shape of {}".format(self.X_test.shape))
            
            return None
        else:
            return sampled
        
    def init_model(self, optimizer='adam', loss='mean_squared_error', kernel_regularizer=None, num_lstm_units = 64, num_lstm_units_2 = 64,num_dense_units = 25, learning_rate=0.001):
        """
        Initializes the LSTM model with the specified optimizer and loss function.

        Args:
            optimizer (str, optional): The optimizer to use. Defaults to 'adam'.
            loss (str, optional): The loss function to use. Defaults to 'mean_squared_error'.
        """
        reset_seeds()
        model = Sequential()
        model.add(LSTM(num_lstm_units, return_sequences=True,\
                       input_shape= (self.X_train.shape[1], self.features_size),\
                      kernel_regularizer=kernel_regularizer, activation = 'relu')) #added activation = "relu"
        model.add(LSTM(num_lstm_units_2, return_sequences=False, kernel_regularizer=kernel_regularizer, activation = 'relu'))
        model.add(Dense(num_dense_units))
        model.add(Dense(1))
        
        
        #opt = Adam(learning_rate=learning_rate) #to remove and default to the code directly below
        
        #if optimizer == 'adam':
            #opt = Adam(learning_rate=learning_rate)
        #elif optimizer == 'SGD':
            #opt = SGD(learning_rate=learning_rate)
        #else:
            #raise ValueError(f"Unsupported optimizer: {optimizer}")
        
        model.compile(optimizer = optimizer, loss = loss) # changed optimizer = optimizer to optimizer = opt
        self.model = model
        
        return None
    
    def fit_model_train(self, batch_size=100, epochs=50, callbacks=None):
        """
        Trains the LSTM model.

        Args:
            batch_size (int, optional): The batch size for training. 
            epochs (int, optional): The number of epochs to train for. 
        """
        self.test_loss_list = []
        self.pred_per_epoch = []
        history_train = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, verbose=1, callbacks = callbacks)
        self.train_loss_list = history_train.history['loss']  # Store loss values in the list 
       
        
    def predict(self):
        """Makes predictions using the trained LSTM model."""
        self.Y_pred = self.model.predict(self.X_test)
        #mse = mean_squared_error(self.Y_test, self.Y_pred)
        #mse_list.append(mse)
        

        
    def key_metrics(self,epoch=None):
        Y_actual = self.Y
        Y_pred = self.Y_pred if epoch == None else self.pred_per_epoch[epoch-1]
        mse = mean_squared_error(self.Y_test, Y_pred)
        mae = mean_absolute_error(self.Y_test, Y_pred)
        exp_ratio = explained_variance_score(self.Y_test, Y_pred)
        
        return Y_actual[-1],(Y_pred[-1]*self.__scale_Y__[1] + self.__scale_Y__[0]), mse, mae, exp_ratio 
    
    def cross_validation(self, n_splits = 5, batch_size=100, epochs=10, callback=False):
        """
        Perform cross-validation for the LSTM model.

        Parameters:
        - n_splits: Number of cross-validation splits (int, default=5)
        - batch_size: Number of samples per gradient update (int, default=100)
        - epochs: Number of epochs to train the model (int, default=50)
        - callback: Flag indicating whether to use custom callback during training (bool, default=True)

        Returns:
        - None
        """
        reset_seeds()
        tscv = TimeSeriesSplit(n_splits=5)
        
        cv_train_scores = []
        cv_val_scores = []
        
        print('Initiate cross-validation')
        i = 1

        for train_index, test_index in tscv.split(self.X_train):
            print(f'Running cross validation round {i}')
            self.temp_val_scores = []

            #print(self.X_train[train_index].shape)
            #print(self.X_train[test_index].shape)
            
            cv_X_train, cv_X_test = self.X_train[train_index], self.X_train[test_index]
            cv_y_train, cv_y_test = self.Y_train[train_index], self.Y_train[test_index]
            
            #print(cv_X_test.shape)
    
            # Fit your model on the training data
            history_train = self.model.fit(cv_X_train, cv_y_train, batch_size=batch_size,\
                                           epochs=epochs,\
                                           verbose = 2)
            cv_train_scores.append(history_train.history['loss'])
            cv_val_scores.append(self.temp_val_scores)
            print('==============================================')
            i+=1
            
        self.cv_train_scores = np.array(cv_train_scores).mean(axis = 0)
        self.cv_val_scores = np.array(cv_val_scores).mean(axis = 0)



In [17]:
sheet_name = ['20220524','20220823','20221122','20230221','20230523']
symbols = pd.read_excel("target_stock_universe.xlsx", sheet_name = sheet_name[0]) # change here to get the other sheet
symbols['Ticker_Symbol'] = symbols['Ticker'].str.split().str[0]+".L"
symbols = symbols[symbols['Ticker_Symbol']!='HSV.L']
symbols = symbols[symbols['Ticker_Symbol']!='HBR.L']

symbols_short = symbols['Ticker_Symbol'] # change here to get the full list of stock
data = price_volume_data(symbols_short.to_list(), '2017-05-25', '2022-05-31')
data.get_technical_indicators()
technical_indicators = ['Close', 'Volume', 'SMA_5', 'SMA_14', 'ATR_5', 'ATR_14', 'ADX_5', 'ADX_14', 'RSI_5','RSI_14', 'MACD', 'MACD_signal', 'BBANDS_upp', 'BBANDS_mid', 'BBANDS_low']

In [20]:
def get_technicals(technical_indicators, symbols_short, data):
    results = {}
    Y = {}
    X = {}
    for x in range(len(symbols_short)):
        results[x] = data.pv[f'{symbols_short.iloc[x]}']
        Y[x] = np.array(results[x]['Close']).reshape(-1,1)
        X[x] = np.array(results[x][technical_indicators])
    return results, Y, X

In [21]:
def run_LSTM(X,Y):
    results = {}
    for x in range(len(symbols_short)):
        model1 = LSTM_predictor(X[x], Y[x])
        model1.prepare_data(train_test_split = 0.8, timestep = 3, look_forward = 3, inplace = True)
        model1.init_model(optimizer='adam', loss='mean_squared_error')
        model1.cross_validation()
        model1.fit_model_train(epochs=30,batch_size=32)
        #return model1
        #model1.fit_model_train(epochs=30,batch_size=32,callbacks=[CustomCallback(model1)])
        model1.predict()
        #return model1
        model1.key_metrics()
        #model1.plot_prediction()
        results[x] = model1.key_metrics()
    return results
    

In [22]:
results, Y, X = get_technicals(technical_indicators, symbols_short, data)

results

model1 = LSTM_predictor(X1, Y1)
model1.prepare_data(train_test_split = 0.8, timestep = 30, look_forward = 3, inplace = True)
model1.init_model(optimizer='adam', loss='mean_squared_error')
model1.cross_validation()

In [94]:
metrics = run_LSTM(X,Y)

[[ 1.32799695e+03  9.29639000e+05  1.29484241e+03  1.27956278e+03
   5.91428859e+01  5.71240291e+01  3.17355431e+01  2.83437264e+01
   6.02717357e+01  4.72830013e+01 -4.99436594e+01 -5.84700716e+01
   1.37312872e+03  1.29484241e+03  1.21655610e+03]
 [ 1.32390942e+03  1.55683200e+06  1.29111821e+03  1.27914105e+03
   5.29460332e+01  5.50550715e+01  2.82768929e+01  2.71937925e+01
   5.88505299e+01  4.68740812e+01 -4.43403955e+01 -5.56441363e+01
   1.36133122e+03  1.29111821e+03  1.22090521e+03]
 [ 1.32481775e+03  1.55405700e+06  1.29602327e+03  1.28034136e+03
   4.98052363e+01  5.37827127e+01  2.65890832e+01  2.59703104e+01
   5.91183007e+01  4.69838067e+01 -3.93726113e+01 -5.23898313e+01
   1.37135467e+03  1.29602327e+03  1.22069186e+03]
 [ 1.42564368e+03  9.85021000e+05  1.33408281e+03  1.29127391e+03
   6.51869272e+01  5.89920683e+01  3.38582188e+01  2.52496881e+01
   7.85161120e+01  5.74815154e+01 -2.69886880e+01 -4.73096027e+01
   1.43594780e+03  1.33408281e+03  1.23221783e+03]
 [ 1

2023-10-10 22:29:16.177944: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [208]
	 [[{{node Placeholder/_11}}]]
2023-10-10 22:29:16.178078: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [208]
	 [[{{node Placeholder/_11}}]]


3/3 - 1s - loss: 1.0097 - 631ms/epoch - 210ms/step
Epoch 2/10
3/3 - 0s - loss: 0.8515 - 8ms/epoch - 3ms/step
Epoch 3/10
3/3 - 0s - loss: 0.6909 - 8ms/epoch - 3ms/step
Epoch 4/10
3/3 - 0s - loss: 0.5167 - 7ms/epoch - 2ms/step
Epoch 5/10
3/3 - 0s - loss: 0.3264 - 7ms/epoch - 2ms/step
Epoch 6/10
3/3 - 0s - loss: 0.1489 - 7ms/epoch - 2ms/step
Epoch 7/10
3/3 - 0s - loss: 0.0659 - 9ms/epoch - 3ms/step
Epoch 8/10
3/3 - 0s - loss: 0.0996 - 7ms/epoch - 2ms/step
Epoch 9/10
3/3 - 0s - loss: 0.0748 - 8ms/epoch - 3ms/step
Epoch 10/10
3/3 - 0s - loss: 0.0246 - 7ms/epoch - 2ms/step
Running cross validation round 2
Epoch 1/10
5/5 - 0s - loss: 0.0239 - 14ms/epoch - 3ms/step
Epoch 2/10
5/5 - 0s - loss: 0.0236 - 14ms/epoch - 3ms/step
Epoch 3/10
5/5 - 0s - loss: 0.0125 - 14ms/epoch - 3ms/step
Epoch 4/10
5/5 - 0s - loss: 0.0137 - 14ms/epoch - 3ms/step
Epoch 5/10
5/5 - 0s - loss: 0.0094 - 14ms/epoch - 3ms/step
Epoch 6/10
5/5 - 0s - loss: 0.0087 - 15ms/epoch - 3ms/step
Epoch 7/10
5/5 - 0s - loss: 0.0078 - 14

2023-10-10 22:29:16.906179: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [411]
	 [[{{node Placeholder/_11}}]]
2023-10-10 22:29:16.906304: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [411]
	 [[{{node Placeholder/_11}}]]
2023-10-10 22:29:17.082056: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [614,

7/7 - 0s - loss: 0.0507 - 20ms/epoch - 3ms/step
Epoch 2/10
7/7 - 0s - loss: 0.0398 - 21ms/epoch - 3ms/step
Epoch 3/10
7/7 - 0s - loss: 0.0274 - 22ms/epoch - 3ms/step
Epoch 4/10
7/7 - 0s - loss: 0.0179 - 21ms/epoch - 3ms/step
Epoch 5/10
7/7 - 0s - loss: 0.0122 - 23ms/epoch - 3ms/step
Epoch 6/10
7/7 - 0s - loss: 0.0097 - 21ms/epoch - 3ms/step
Epoch 7/10
7/7 - 0s - loss: 0.0087 - 29ms/epoch - 4ms/step
Epoch 8/10
7/7 - 0s - loss: 0.0079 - 22ms/epoch - 3ms/step
Epoch 9/10
7/7 - 0s - loss: 0.0073 - 22ms/epoch - 3ms/step
Epoch 10/10
7/7 - 0s - loss: 0.0068 - 22ms/epoch - 3ms/step
Running cross validation round 4
Epoch 1/10
9/9 - 0s - loss: 0.0282 - 30ms/epoch - 3ms/step
Epoch 2/10
9/9 - 0s - loss: 0.0169 - 28ms/epoch - 3ms/step
Epoch 3/10
9/9 - 0s - loss: 0.0150 - 28ms/epoch - 3ms/step
Epoch 4/10
9/9 - 0s - loss: 0.0126 - 27ms/epoch - 3ms/step
Epoch 5/10
9/9 - 0s - loss: 0.0115 - 27ms/epoch - 3ms/step
Epoch 6/10
9/9 - 0s - loss: 0.0098 - 28ms/epoch - 3ms/step
Epoch 7/10


...

2023-10-10 22:31:35.922487: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1020]
	 [[{{node Placeholder/_11}}]]
2023-10-10 22:31:35.922610: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1020]
	 [[{{node Placeholder/_11}}]]


11/11 - 0s - loss: 0.0529 - 38ms/epoch - 3ms/step
Epoch 7/10
11/11 - 0s - loss: 0.0496 - 34ms/epoch - 3ms/step
Epoch 8/10
11/11 - 0s - loss: 0.0489 - 35ms/epoch - 3ms/step
Epoch 9/10
11/11 - 0s - loss: 0.0484 - 35ms/epoch - 3ms/step
Epoch 10/10
11/11 - 0s - loss: 0.0456 - 35ms/epoch - 3ms/step
Epoch 1/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0602
Epoch 2/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0606
Epoch 3/30
 1/39 [..............................] - ETA: 0s - loss: 0.0385

2023-10-10 22:31:36.309280: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1223,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 22:31:36.309404: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1223]
	 [[{{node Placeholder/_11}}]]


39/39 [==============================] - 0s 2ms/step - loss: 0.0510
Epoch 4/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 5/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0484
Epoch 6/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0464
Epoch 7/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0455
Epoch 8/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0447
Epoch 9/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0450
Epoch 10/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0413
Epoch 11/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0388
Epoch 12/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0349
Epoch 13/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0328
Epoch 14/30
39/39 [==============================] - 0s 2ms/step - loss: 0.0324
Epoch 15/30
39/39 [==============================] - 0s 2m

2023-10-10 22:31:38.653247: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 22:31:38.653350: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]


In [108]:
metrics_copy = pd.DataFrame(metrics)

In [109]:
metrics_copy.columns = symbols_short.to_list()

In [110]:
metrics_copy = metrics_copy.set_index(pd.Index(['Actual Price', 'Predicted Price', 'MSE','MAE','R2']))

In [111]:
metrics_copy

,ICP.L,ABDN.L,SMDS.L,HL.L,DPH.L,HIK.L,HWDN.L,IDS.L,ITV.L,CCL.L,...,DLG.L,EMG.L,DPLM.L,IGG.L,WIZZ.L,FRAS.L,TATE.L,BEZ.L,PNN.L,RSW.L
Actual Price,1464.248291,175.403336,282.856812,824.121399,3567.611328,1640.628174,660.338989,313.700653,65.962585,1011.5,...,250.364975,240.397629,2553.907471,652.19873,3094.0,695.0,722.515076,473.095398,921.890503,4005.439697
Predicted Price,[1264.2468],[163.5097],[267.72086],[752.5042],[3383.6902],[1627.581],[637.03265],[276.65097],[66.964294],[1102.1017],...,[232.5869],[223.5084],[2410.2227],[628.6662],[3054.6003],[648.7346],[747.0776],[459.9702],[978.6994],[3899.5273]
MSE,0.23623,0.077741,0.095043,0.084401,0.043994,0.00076,0.022085,0.108747,0.002242,0.004354,...,1.044022,0.285668,0.045431,0.03513,0.002874,0.101715,0.113386,0.027568,0.094318,0.019304
MAE,0.486035,0.278821,0.30829,0.290519,0.209748,0.027571,0.148612,0.329769,0.047346,0.065982,...,1.021774,0.534479,0.213145,0.187431,0.05361,0.318928,0.336728,0.166035,0.307112,0.13894
R2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [112]:
shares_outstanding = pd.DataFrame(data.shares)

In [113]:
shares_outstanding = shares_outstanding.T

In [114]:
shares_outstanding.columns = symbols_short.to_list()

In [115]:
shares_outstanding

,ICP.L,ABDN.L,SMDS.L,HL.L,DPH.L,HIK.L,HWDN.L,IDS.L,ITV.L,CCL.L,...,DLG.L,EMG.L,DPLM.L,IGG.L,WIZZ.L,FRAS.L,TATE.L,BEZ.L,PNN.L,RSW.L
0,290612992,1899229952,1377449984,474319008,113888000,221079008,548673024,956193024,4034409984,146260000,...,1311399936,1199859968,134034000,395398016,103335000,454926016,401665984,672462976,261327008,72788496


In [116]:
combined = pd.concat([metrics_copy,shares_outstanding], axis=0, join='outer')

In [118]:
combined = combined.T

In [121]:
combined

,Actual Price,Predicted Price,MSE,MAE,R2,0
ICP.L,1464.248291,[1264.2468],0.23623,0.486035,1.0,290612992
ABDN.L,175.403336,[163.5097],0.077741,0.278821,1.0,1899229952
SMDS.L,282.856812,[267.72086],0.095043,0.30829,1.0,1377449984
HL.L,824.121399,[752.5042],0.084401,0.290519,1.0,474319008
DPH.L,3567.611328,[3383.6902],0.043994,0.209748,1.0,113888000
HIK.L,1640.628174,[1627.581],0.00076,0.027571,1.0,221079008
HWDN.L,660.338989,[637.03265],0.022085,0.148612,1.0,548673024
IDS.L,313.700653,[276.65097],0.108747,0.329769,1.0,956193024
ITV.L,65.962585,[66.964294],0.002242,0.047346,1.0,4034409984
CCL.L,1011.5,[1102.1017],0.004354,0.065982,1.0,146260000


In [123]:
combined['final_marketcap'] = combined[0]*combined['Actual Price']

In [129]:
combined = combined.rename(columns={0: 'Market Share'})

In [131]:
combined

,Actual Price,Predicted Price,MSE,MAE,R2,Market Share,final_marketcap
ICP.L,1464.248291,[1264.2468],0.23623,0.486035,1.0,290612992,425529576882.9375
ABDN.L,175.403336,[163.5097],0.077741,0.278821,1.0,1899229952,333131268597.699219
SMDS.L,282.856812,[267.72086],0.095043,0.30829,1.0,1377449984,389621110507.25
HL.L,824.121399,[752.5042],0.084401,0.290519,1.0,474319008,390896444410.048828
DPH.L,3567.611328,[3383.6902],0.043994,0.209748,1.0,113888000,406308118937.5
HIK.L,1640.628174,[1627.581],0.00076,0.027571,1.0,221079008,362708449166.773438
HWDN.L,660.338989,[637.03265],0.022085,0.148612,1.0,548673024,362310190101.1875
IDS.L,313.700653,[276.65097],0.108747,0.329769,1.0,956193024,299958376095.679688
ITV.L,65.962585,[66.964294],0.002242,0.047346,1.0,4034409984,266120113306.78125
CCL.L,1011.5,[1102.1017],0.004354,0.065982,1.0,146260000,147941990000.0


In [132]:
sorted_df = combined.sort_values(by='final_marketcap')

In [133]:
sorted_df

,Actual Price,Predicted Price,MSE,MAE,R2,Market Share,final_marketcap
CCL.L,1011.5,[1102.1017],0.004354,0.065982,1.0,146260000,147941990000.0
PNN.L,921.890503,[978.6994],0.094318,0.307112,1.0,261327008,240914886834.230469
IGG.L,652.19873,[628.6662],0.03513,0.187431,1.0,395398016,257878084065.0625
ITV.L,65.962585,[66.964294],0.002242,0.047346,1.0,4034409984,266120113306.78125
INVP.L,433.472778,[401.9024],0.167294,0.409016,1.0,645161984,279660157671.125
EMG.L,240.397629,[223.5084],0.285668,0.534479,1.0,1199859968,288443491180.261719
TATE.L,722.515076,[747.0776],0.113386,0.336728,1.0,401665984,290209728829.285156
RSW.L,4005.439697,[3899.5273],0.019304,0.13894,1.0,72788496,291549931382.660156
IDS.L,313.700653,[276.65097],0.108747,0.329769,1.0,956193024,299958376095.679688
SXS.L,2927.939453,[2899.7961],0.00301,0.054867,1.0,102758000,300869202324.21875


In [134]:
sorted_df.to_csv("results_stocks_comparison.csv")

In [14]:
#################